In [ ]:
!pip install datasets

In [ ]:
!pip install datasets --upgrade --quiet
from huggingface_hub import login
login()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import pickle
from datasets import load_dataset
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset,Dataset
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm

In [ ]:
dataset=load_dataset("CohleM/english-to-nepali")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/328 [00:00<?, ?B/s]

(…)-00000-of-00001-ea91b3ffe2804196.parquet:   0%|          | 0.00/28.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/177334 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['en', 'ne'],
        num_rows: 177334
    })
})

In [ ]:
dataset['train']['en'][0],dataset['train']['ne'][0]

('Technical committees will be attached to each ministry.',
 'प्रत्येक मन्त्रालय अन्तर्गत शिल्प (टेक्निकल) कमिटीहरु गठन गरिनेछन्')

In [ ]:
english=dataset['train']['en'][:20000]
nepali=dataset['train']['ne'][:20000]

In [ ]:
english[:5]

['Technical committees will be attached to each ministry.',
 'An Administrative Panel will examine and advise the Cabinet Development Committee regarding all proposals having to do with terms of employment or administrative rules and regulations applicable to personnel engaged in Five –Year Plan operations, procedures or delegations of authority.',
 'A special committee will be appointed to advise on the coordination of foreign aid.',
 'In districts where development is more intensive, the Badahakims  will be assisted by District Development Officer and staffs of technical advisors appointed by executive agencies of the central Government.',
 "About three fourths of Nepal's 15 crores per year imports from India consist of textiles, cigarettes, sugar, petrol, etc., on which products excise taxes within India average about 20 per cent."]

In [ ]:
english=' '.join(english)

In [ ]:
english

'Technical committees will be attached to each ministry. An Administrative Panel will examine and advise the Cabinet Development Committee regarding all proposals having to do with terms of employment or administrative rules and regulations applicable to personnel engaged in Five –Year Plan operations, procedures or delegations of authority. A special committee will be appointed to advise on the coordination of foreign aid. In districts where development is more intensive, the Badahakims  will be assisted by District Development Officer and staffs of technical advisors appointed by executive agencies of the central Government. About three fourths of Nepal\'s 15 crores per year imports from India consist of textiles, cigarettes, sugar, petrol, etc., on which products excise taxes within India average about 20 per cent. As indicated above, the entire Five-Year Plan as projected calls for an outlay of approximately 33 crore rupees. Full execution of the Plan would require in addition to t

In [ ]:
list("aayush".encode("utf-8"))

[97, 97, 121, 117, 115, 104]

In [ ]:
class Tokenizer:
    def __init__(self):
      self.vocab={}

    def load(self,path):
      with open(path, "rb") as file:
        self.vocab=pickle.load(file)

    def train(self,text, vocab_size=1000):
        tokens = list(text.encode("utf-8"))
        vocab_size = vocab_size
        num_merges = vocab_size - 256
        merges = {}
        ids = list(tokens)

        for i in tqdm(range(num_merges), desc="Training BPE tokenizer"):
            stats = self.get_stats(ids)
            pair = max(stats, key=stats.get)
            idx = 256 + i
            ids = self.merge(ids, pair, idx)
            merges[pair] = idx

        self.vocab['vocab'] = {idx: bytes([idx]) for idx in range(256)}
        for (p0, p1), idx in merges.items():
            self.vocab['vocab'][idx] = self.vocab['vocab'][p0] + self.vocab['vocab'][p1]

        self.vocab['merges']=merges

        return self.vocab

    def get_stats(self, ids):
        counts = {}
        for pair in zip(ids, ids[1:]):
            counts[pair] = counts.get(pair, 0) + 1
        return counts

    def merge(self, ids, pair, idx):
        new_ids = []
        i = 0
        while i < len(ids):
            if i < len(ids) - 1 and ids[i] == pair[0] and ids[i + 1] == pair[1]:
                new_ids.append(idx)
                i += 2
            else:
                new_ids.append(ids[i])
                i += 1
        return new_ids

    def decode(self, ids):
        token = b"".join(self.vocab['vocab'][idx] for idx in ids)
        text = token.decode("utf-8", errors="replace")
        return text

    def encode(self, text):
        token = list(text.encode("utf-8"))
        while len(token) >= 2:
            stats = self.get_stats(token)
            pair = min(stats, key=lambda p: self.vocab['merges'].get(p, float("inf")))
            if pair not in self.vocab['merges']:
                break
            idx = self.vocab['merges'][pair]
            token = self.merge(token, pair, idx)
        return token


In [ ]:
en_tokenizer=Tokenizer()
np_tokenizer=Tokenizer()

In [ ]:
en_tokenizer.train(english)

Training BPE tokenizer: 100%|██████████| 744/744 [06:31<00:00,  1.90it/s]


{'vocab': {0: b'\x00',
  1: b'\x01',
  2: b'\x02',
  3: b'\x03',
  4: b'\x04',
  5: b'\x05',
  6: b'\x06',
  7: b'\x07',
  8: b'\x08',
  9: b'\t',
  10: b'\n',
  11: b'\x0b',
  12: b'\x0c',
  13: b'\r',
  14: b'\x0e',
  15: b'\x0f',
  16: b'\x10',
  17: b'\x11',
  18: b'\x12',
  19: b'\x13',
  20: b'\x14',
  21: b'\x15',
  22: b'\x16',
  23: b'\x17',
  24: b'\x18',
  25: b'\x19',
  26: b'\x1a',
  27: b'\x1b',
  28: b'\x1c',
  29: b'\x1d',
  30: b'\x1e',
  31: b'\x1f',
  32: b' ',
  33: b'!',
  34: b'"',
  35: b'#',
  36: b'$',
  37: b'%',
  38: b'&',
  39: b"'",
  40: b'(',
  41: b')',
  42: b'*',
  43: b'+',
  44: b',',
  45: b'-',
  46: b'.',
  47: b'/',
  48: b'0',
  49: b'1',
  50: b'2',
  51: b'3',
  52: b'4',
  53: b'5',
  54: b'6',
  55: b'7',
  56: b'8',
  57: b'9',
  58: b':',
  59: b';',
  60: b'<',
  61: b'=',
  62: b'>',
  63: b'?',
  64: b'@',
  65: b'A',
  66: b'B',
  67: b'C',
  68: b'D',
  69: b'E',
  70: b'F',
  71: b'G',
  72: b'H',
  73: b'I',
  74: b'J',
  75: b'K',

In [ ]:
en_tokenizer.encode("my name is Aayush Dangol")

[887, 110, 395, 256, 310, 65, 474, 403, 340, 68, 647, 336]

In [ ]:
en_tokenizer.decode([887, 110, 395, 256, 310, 65, 474, 403, 340, 68, 647, 336])

'my name is Aayush Dangol'

In [ ]:
import pickle

with open("data.pkl", "wb") as f:
    pickle.dump(en_tokenizer.vocab, f)


In [ ]:
en_tokenizer.load('data.pkl')

In [ ]:
en_tokenizer.vocab

{'vocab': {0: b'\x00',
  1: b'\x01',
  2: b'\x02',
  3: b'\x03',
  4: b'\x04',
  5: b'\x05',
  6: b'\x06',
  7: b'\x07',
  8: b'\x08',
  9: b'\t',
  10: b'\n',
  11: b'\x0b',
  12: b'\x0c',
  13: b'\r',
  14: b'\x0e',
  15: b'\x0f',
  16: b'\x10',
  17: b'\x11',
  18: b'\x12',
  19: b'\x13',
  20: b'\x14',
  21: b'\x15',
  22: b'\x16',
  23: b'\x17',
  24: b'\x18',
  25: b'\x19',
  26: b'\x1a',
  27: b'\x1b',
  28: b'\x1c',
  29: b'\x1d',
  30: b'\x1e',
  31: b'\x1f',
  32: b' ',
  33: b'!',
  34: b'"',
  35: b'#',
  36: b'$',
  37: b'%',
  38: b'&',
  39: b"'",
  40: b'(',
  41: b')',
  42: b'*',
  43: b'+',
  44: b',',
  45: b'-',
  46: b'.',
  47: b'/',
  48: b'0',
  49: b'1',
  50: b'2',
  51: b'3',
  52: b'4',
  53: b'5',
  54: b'6',
  55: b'7',
  56: b'8',
  57: b'9',
  58: b':',
  59: b';',
  60: b'<',
  61: b'=',
  62: b'>',
  63: b'?',
  64: b'@',
  65: b'A',
  66: b'B',
  67: b'C',
  68: b'D',
  69: b'E',
  70: b'F',
  71: b'G',
  72: b'H',
  73: b'I',
  74: b'J',
  75: b'K',

In [ ]:
en_tokenizer.load("/content/drive/MyDrive/English_Nepali/English_Tokenizer_500.pkl")

In [ ]:
text="i love football"
en_tokenizer.encode(text)

[105, 32, 461, 319, 908, 591, 98, 475]